In [1]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np
import joblib
import os

# Initialize the main window
root = tk.Tk()
root.title("Predicting Discharge Coefficient of PKW")
root.geometry("1500x415")
root.configure(bg='#FFFFFF')  # Set the background color to white

# Create the main frame
frame = tk.Frame(root, padx=3, pady=20, bg='#FFFFFF')
frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# Create the right panel for the image
right_panel = tk.Frame(root, bg='#FFFFFF')
right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Load the image from the desktop and create an image on the canvas using PIL
image_path = r"C:\Users\asus1\Desktop\image1.jpeg"  # Use the provided file path

# Load the image and create an image label
try:
    image = Image.open(image_path)
    image = image.resize((400, 335), Image.LANCZOS)  # Resize the image to fit the panel
    root.image_image = ImageTk.PhotoImage(image)  # Store the image
    image_label = tk.Label(right_panel, image=root.image_image, bg='#FFFFFF')
    image_label.pack(padx=10, pady=20)
except Exception as e:
    messagebox.showerror("Error", f"Failed to load image: {e}")

# Check if the CatBoost model file exists
model_path_catb = r"C:\Users\asus1\Desktop\CTB.joblib"  # Use the provided file path
if not os.path.exists(model_path_catb):
    messagebox.showerror("Error", "CatBoost model file not found.")
    raise SystemExit  # Exit if the model file is not found

# Load the CatBoost model
model_catb = joblib.load(model_path_catb)

# Title and information labels
label_title = tk.Label(frame, text='Predicting Discharge Coefficient of Piano Key Weir (PKW)', font=('Courier New', 17, 'bold', 'underline'), fg='#C00000', bg='#FFFFFF')
label_title.grid(row=0, column=0, columnspan=3, pady=10, sticky='w')

label_developer = tk.Label(frame, text='🌟Developers: Mohamed K. Elshaarawy (0000-0002-1793-5617) & Abdelrahman K. Hamed (0000-0002-8501-3673)', font=('Courier New', 12, 'bold'), fg='#00008B', bg='#FFFFFF')
label_developer.grid(row=1, column=0, columnspan=3, pady=10, sticky='w')

label_inputdefinetitle = tk.Label(frame, text='- Definition of Inputs', font=('Courier New', 16, 'bold'), fg='#000000', bg='#FFFFFF')
label_inputdefinetitle.grid(row=3, column=0, columnspan=3, pady=10, sticky='w')


# Function to create labels and entry fields together
def create_label_entry(frame, text, row, col):
    label_entry_frame = tk.Frame(frame, bg='#FFFFFF')
    label_entry_frame.grid(row=row, column=col, columnspan=2, padx=10, pady=10, sticky='w')
    label = tk.Label(label_entry_frame, text=text, font=('Courier New', 14,'bold'), fg='#006600', bg='#FFFFFF', anchor='w')
    label.pack(side=tk.LEFT)
    entry = tk.Entry(label_entry_frame, font=('Courier New', 16, 'bold'), bg='#FFFFFF', highlightbackground='#000000', highlightthickness=1, bd=0, width=10)
    entry.pack(side=tk.LEFT, padx=10)
    return entry

# Input labels and entry fields
entries = {}
param_labels = [
    'X1: PKW key widths ratio (Wi/Wo)',
    'X2: PKW height ratio (P/Wu)',
    'X3: Total upstream head ratio (H/Wu)'
]
for i, text in enumerate(param_labels):
    entries[text] = create_label_entry(frame, text, i+4, 0)

# Prediction result labels
label_output = tk.Label(frame, text='- Discharge Coefficient (Cd) = ', font=('Courier New', 16, 'bold'), fg='#C00000', bg='#FFFFFF')
label_output.grid(row=12, column=0, pady=10, sticky='w')

output_value = tk.Label(frame, text="0.0", font=('Courier New', 16, 'bold'), fg='#C00000', relief="sunken", width=15, bg='#FFFFFF')
output_value.grid(row=12, column=1, pady=10)

# Function to clear all entries
def clear_entries():
    for entry in entries.values():
        entry.delete(0, tk.END)
        entry.insert(0, "0.0")
    output_value.config(text="0.0")

# Function to predict the result
def predict():
    try:
        input_values = [float(entries[label].get()) for label in param_labels]
        input_data = np.array([input_values])
        
        # Predict using CatBoost model
        prediction = model_catb.predict(input_data)
        
        # Display the predictions
        output_value.config(text=f"{prediction[0]:.4f}")
        output_value.config(background='yellow', relief='solid', borderwidth=1)
    except ValueError:
        messagebox.showerror("Error", "Invalid input. Please enter numeric values.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Buttons
button_frame = tk.Frame(frame, bg='#FFFFFF')
button_frame.grid(row=11, column=0, columnspan=3, pady=10)

clear_button = tk.Button(button_frame, text="Clear", font=('Courier New', 14), command=clear_entries, bg='#FFFFFF')
clear_button.grid(row=0, column=0, padx=10)
predict_button = tk.Button(button_frame, text="Predict", font=('Courier New', 14), command=predict, bg='#FFFFFF')
predict_button.grid(row=0, column=1, padx=10)

# Initial values for entries
clear_entries()

# Run the main loop
root.mainloop()
